In [ ]:
import sys
!{sys.executable} -m pip install airstack

In [ ]:
import asyncio
import json
from airstack.execute_query import AirstackClient

In [ ]:
api_client = AirstackClient(api_key='9deb75ae44b24f4ca9eecfbc37492edd')

In [ ]:
query =  """
query CurrentTokenBalances($identity: Identity) {
  Ethereum: TokenBalances(
    input: {filter: {owner: {_eq: $identity}}, blockchain: ethereum, order: {lastUpdatedTimestamp: DESC}, limit: 200}
  ) {
    TokenBalance {
      amount
      tokenType
      tokenAddress
      token {
        name
      }
    }
    pageInfo {
      nextCursor
      prevCursor
    }
  }
  Polygon: TokenBalances(
    input: {filter: {owner: {_eq: $identity}}, blockchain: polygon, order: {lastUpdatedTimestamp: DESC}, limit: 200}
  ) {
    TokenBalance {
      amount
      tokenType
      tokenAddress
      token {
        name
      }
    }
    pageInfo {
      nextCursor
      prevCursor
    }
  }
}
  """
  
variables = {
  "identity": "vitalik.eth"
}

In [ ]:
execute_query_client = api_client.create_execute_query_object(query=query, variables=variables)
query_response = await execute_query_client.execute_paginated_query()

print("error", query_response.error)
print("data ", json.dumps(query_response.data))

In [ ]:
if query_response.has_next_page: 
    query_response = await query_response.get_next_page
    print("page 2 data ", json.dumps(query_response.data))
    
if query_response.has_prev_page: 
    query_response = await query_response.get_prev_page
    print("page 1 data ", json.dumps(query_response.data))

In [ ]:
async def execute_with_pagination(api_client, query, variables):
    execute_query_client = api_client.create_execute_query_object(query=query, variables=variables)
    query_response = await execute_query_client.execute_paginated_query()
    
    while query_response.has_next_page:
            yield query_response
            query_response = await query_response.get_next_page
    yield query_response

In [ ]:
responses =  execute_with_pagination(api_client=api_client,query=query, variables=variables)

async for response in responses:
    print(response.data)